# Kernel 1: Naive Kernel
So, this is where it begins. The descent into madness.

So now that are familiar with basics of how the execution model works, where and how a CUDA kernel is launched, and how its result is recevied by the CPU, let's take a look at the actual first (and worst) kernel for General Matrix-Matrix Multiplication.

Small easter egg, the kernel we first launched in the previous notebook is the one we will be defining now, so the arguments we were arbitrarily passing before will make more sense now.


The actual kernel invocation + execution configration parameters set-up:

In [ ]:
// create as many blocks as necessary to map all of C
dim3 gridDim(CEIL_DIV(M, 32), CEIL_DIV(N, 32), 1);  

// 32 * 32 = 1024 threads per block
dim3 blockDim(32, 32, 1);

sgemm_naive<<<gridDim, blockDim>>>(M, N, K, alpha, A, B, beta, C);


The way our execution model works here, is that we assign each thread one entry of our resuilt matrix C, essentially giving each mini worker one allocated spot they must compute and fill in. Based on the block dimension we defined, we have 1024 threads per block, which means each block covers 1024 entries of the matrix. This essentially means that we need total dimensions of the matrix / 1024 blocks to get all the computation needed.

Assume we know that the dimensions of our output matrix C are M x N (will be illuminated later). 

If you're thinking (I did initially) it'd be equivalent to just have gridDim( CEIL_DIV(M, 1024), N, 1); since that will also give us the same number of blocks, unfortunately, that won't work. This is as the layout of the blocks over the 2D matrix must match our thread indexing math - which I will talk about in the next code snippet, finally looking into the black box of our kernel.


In [ ]:
_global__ void sgemm_naive(int M, int N, int K, float alpha, const float* A, const float* B, float beta, float* C) {


// compute position in C that this thread is responsible for
const uint col = blockIdx.x * blockDim.x + threadIdx.x;         // threadIdx.x and threadIdx.y will vary from 0 to 31 
const uint row = blockIdx.y * blockDim.y + threadIdx.y;         // blockIdx.x and blockIdx.y will vary from 0 to CEIL_DIV(N,32) and CEIL_DIV(M,32) respectively

...
}

So as you can see, a kernel is not too far off from regular functions we are used to, having the name defined with the list of arugments. The __global__ keyword you're going to be seeing a lot of is a function qualifier that tells the compiler that the function is not any regular old function, it's a **kernel** - launched from the host (CPU) code but ran on the device (GPU). It then allows us to call the kernel from CPU code, using that <<< >>> triple-chevron launch syntax you should be familiar with at this point.

Kernels always have a return type of void, since they can't return values directly. Instead, they write results to GPU memory, which can be copied back to CPU memory using cudaMemcpy(...). I hope these recurring concepts and keywords are steadily working their way into your mental muscle memory, building a natural familiarity with CUDA.

#### Thread Numbering
Now, above I mentioned, that we couldn't do a less sophisticated approach to the grid dimensions because of thread numbering that we see above. This was something that took me a while to get my head around so I will do you the favour of avoiding a stiff neck by explaining it in a way that clicked to me - and then how it ties in to the dimension defintion above.

**Key Concept**: CUDA code is written from the perspective of a single thread. This is not just an attempt to personify our mini workers, rather, this means that the code is executed independently (and uniquely) by each thread, and built-in variables such as blockIdx, and threadIdx will hold different values depending on which thread is running it.

Since we defined our blockDim.x and blockDim.y both as being 32, our threadId.x and threadId.y are the corresponding threads essentially also holding 32 possible values - ranging from 0 to 31 ~ allowing the 1024 threads to have unique a (threadId.x,threadId.y)

Similarly, based on our gridDim definition, blockId.x and blockId.y will vary from 0 to M/32 and N/32, respectively.

Using these thread-specific, runtime-determined variables i.e. threadIdx telling us where a thread sits in a block and blockIdx telling us which block it belongs to in a grid, combined with the known dimentions of each block, we can calculate a global index for the thread ~ essentially assign it a unique entry in our result matrix that it is responsible for computing.

If you are familiar with accessing using pointer arithmetic, this will feel a lot similar to stuff you have worked with before.

We are essentially trying to find the exact (row, column) index on the result matrix for each thread.

### Another great image from Simon

![](../../images/GEMM1/naivekernel.png)

This image, as effective as it is, might be a bit overwhelming to intially get the message across, so I invite the user to look at this slightly simpler image included with a as-simple example, and then circle back to the more detailed, <> one above.

example: Let's say our output matrix is of dimension M = 4 x N = 6, so here.



